# S&P 500 Signal Classifier

In [ ]:
Approach:
1. Create features
2. Winsorize data
3. Test, Train split data
4. Scale the test and train data
6. Original, Oversample, and Undersample the data
7. Apply RandomForestClassifier, XGBoost, and LightBoost to predict

## Install Required Packages

In [ ]:
# Install Yahoo Finance
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
# Install SciKit Learn
!pip install scikit-learn

In [ ]:
# Install XGBoost
!pip install xgboost

In [ ]:
# Install LightBoost
!pip install lightgbm

## Import Required Dependencies

In [1]:
# Import the required libraries and dependencies
import yfinance as yf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from xgboost import XGBClassifier
from xgboost import plot_importance
import lightgbm as lgb

In [ ]:
import warnings
# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

## Retrieve and Load Data

In [ ]:
# Retreive data
sp500 = yf.Ticker('^GSPC')
sp500 = sp500.history(period='max')

In [ ]:
# Drop timezone from datetime
sp500 = sp500.reset_index()
sp500['Date'] = sp500['Date'].dt.tz_localize(None)
sp500.set_index('Date', inplace=True)

In [ ]:
# Get the last rows of the DataFrame
sp500 = sp500.tail(24252) # Get max available data

In [ ]:
# Display data
sp500

In [ ]:
# Create stock df
stock_df = pd.DataFrame(sp500).dropna()
stock_df

In [ ]:
# Drop columns
stock_df.drop(columns={'Volume', 'Dividends', 'Stock Splits'}, inplace=True)

# Sort by ascending date
stock_df = stock_df.sort_values(by="Date", ascending=True)

# Review the first and last five rows of the DataFrame
display(stock_df.head())
display(stock_df.tail())

In [ ]:
# Rename dataframe to data
data = stock_df

## Perform Feature Engineering

In [ ]:
# Feature Engineering
# Moving Averages
data['SMA10'] = data['Close'].rolling(window=10).mean()
data['SMA20'] = data['Close'].rolling(window=20).mean()
data['SMA30'] = data['Close'].rolling(window=20).mean()
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA100'] = data['Close'].rolling(window=100).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()

# RSI
def rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = rsi(data)

# MACD
data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']
data['SL'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
data['BollM'] = data['Close'].rolling(window=20).mean()
data['BollU'] = data['BollM'] + (2 * data['Close'].rolling(window=20).std())
data['BollL'] = data['BollM'] - (2 * data['Close'].rolling(window=20).std())

# Identify crossover signal for Target Column
# Close < Bollinger Mid and RSI < 50 and MACD < 0
data['Signal'] = np.where((data['Close'] < data['BollM']) & (data['RSI'] < 50) & (data['MACD'] < 0), 1, 0)

# Calculate return
data['Return'] = data['Close'].pct_change().shift(-1)  # Next day's return

# Label Data
# Buy ONLY if all Crossover conditions are met
# Assume profitable if return > 0%
data['Target'] = np.where((data['Signal'] > 0) & (data['Return'] > 0.0), 1, 0)

# Prepare dataset
features = ['SMA10', 'SMA20', 'SMA30', 'SMA50', 'SMA100', 'SMA200', 'Signal', 'RSI', 'MACD', 'SL', 'BollM', 'BollU', 'BollL']
dataset = data.dropna()[features + ['Target']]

## Perform Feature Engineering

In [ ]:
# Feature Engineering
# Moving Averages
data['SMA10'] = data['Close'].rolling(window=10).mean()
data['SMA20'] = data['Close'].rolling(window=20).mean()
data['SMA30'] = data['Close'].rolling(window=20).mean()
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA100'] = data['Close'].rolling(window=100).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()

# RSI
def rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = rsi(data)

# MACD
data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']
data['SL'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
data['BollM'] = data['Close'].rolling(window=20).mean()
data['BollU'] = data['BollM'] + (2 * data['Close'].rolling(window=20).std())
data['BollL'] = data['BollM'] - (2 * data['Close'].rolling(window=20).std())

# Identify crossover signal for Target Column
# Close < Bollinger Mid and RSI < 50 and MACD < 0
data['Signal'] = np.where((data['Close'] < data['BollM']) & (data['RSI'] < 50) & (data['MACD'] < 0), 1, 0)

# Calculate return
data['Return'] = data['Close'].pct_change().shift(-1)  # Next day's return

# Label Data
# Buy ONLY if all Crossover conditions are met
# Assume profitable if return > 0%
data['Target'] = np.where((data['Signal'] > 0) & (data['Return'] > 0.0), 1, 0)

# Prepare dataset
features = ['SMA10', 'SMA20', 'SMA30', 'SMA50', 'SMA100', 'SMA200', 'Signal', 'RSI', 'MACD', 'SL', 'BollM', 'BollU', 'BollL']
dataset = data.dropna()[features + ['Target']]

## Perform Winsorization of Data

In [ ]:
from scipy.stats.mstats import winsorize
data['Open'] = winsorize(np.array(data['Open']), limits=[.05, .05])
data['High'] = winsorize(np.array(data['High']), limits=[.05, .05])
data['Low'] = winsorize(np.array(data['Low']), limits=[.05, .05])
data['Close'] = winsorize(np.array(data['Close']), limits=[.05, .05])
data['SMA10'] = winsorize(np.array(data['SMA10']), limits=[.05, .05])
data['SMA20'] = winsorize(np.array(data['SMA20']), limits=[.05, .05])
data['SMA30'] = winsorize(np.array(data['SMA30']), limits=[.05, .05])
data['SMA50'] = winsorize(np.array(data['SMA50']), limits=[.05, .05])
data['SMA100'] = winsorize(np.array(data['SMA100']), limits=[.05, .05])
data['SMA200'] = winsorize(np.array(data['SMA200']), limits=[.05, .05])
data['EMA12'] = winsorize(np.array(data['EMA12']), limits=[.05, .05])
data['EMA26'] = winsorize(np.array(data['EMA26']), limits=[.05, .05])
data['MACD'] = winsorize(np.array(data['MACD']), limits=[.05, .05])
data['BollM'] = winsorize(np.array(data['BollM']), limits=[.05, .05])
data['BollU'] = winsorize(np.array(data['BollU']), limits=[.05, .05])
data['BollL'] = winsorize(np.array(data['BollL']), limits=[.05, .05])
data['SL'] = winsorize(np.array(data['SL']), limits=[.05, .05])

In [ ]:
plt.figure(figsize=(20, 6))
sns.boxplot(data=data)
plt.title("Boxplot for Outlier Detection - Train")
plt.show()

## Perform Train and Test Splits

In [ ]:
# Define train and test sets
X = dataset[features]
y = dataset['Target']

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

## Perform Scaling

In [ ]:
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled arrays back to DataFrames to maintain column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=features, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=features, index=X_test.index)

# Perform Classifications

## Original Scaled Data with Untuned RandomForestClassifier

In [ ]:
# Define the model
clf = RandomForestClassifier(random_state=1)

# Fit the model
clf.fit(X_train_scaled, y_train)

# Predict using the model
train_pred = clf.predict(X_train_scaled)
test_pred = clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Over Sampled Scaled Data with Untuned RandomForestClassifier

In [ ]:
# Import RandomOverSampler from imblearn
from imblearn.over_sampling import RandomOverSampler

# Instantiate a RandomOversampler instance
ros = RandomOverSampler(random_state=1)

# Fit the training data to the `RandomOverSampler` model
X_oversampled, y_oversampled = ros.fit_resample(X_train_scaled, y_train)

# Count distinct values
y_oversampled.value_counts()

In [ ]:
# Instantiate the model
ros_clf = RandomForestClassifier(random_state=1)

# Fit the model
ros_clf.fit(X_oversampled, y_oversampled)

# Predict using the model
train_pred = ros_clf.predict(X_train_scaled)
test_pred = ros_clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Under Sampled Scaled Data with Untuned RandomForestClassifier

In [ ]:
# Import RandomUnderSampler from imblearn
from imblearn.under_sampling import RandomUnderSampler

# Instantiate a RandomOversampler instance
rus = RandomUnderSampler(random_state=1)

# Fit the training data to the `RandomOverSampler` model
X_undersampled, y_undersampled = rus.fit_resample(X_train_scaled, y_train)

# Count distinct values
y_undersampled.value_counts()

In [ ]:
# Instantiate the model
rus_clf = RandomForestClassifier(random_state=1)

# Fit the model
rus_clf.fit(X_undersampled, y_undersampled)

# Predict using the model
train_pred = rus_clf.predict(X_train_scaled)
test_pred = rus_clf.predict(X_test_scaled)

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Original Scaled Data and Untuned XGBoost Classifier

In [ ]:
# Define the model
xgb_clf = XGBClassifier(random_state=1)

# Fit the model
xgb_clf.fit(X_train_scaled, y_train)

# Predict using the model
train_pred = xgb_clf.predict(X_train_scaled)
test_pred = xgb_clf.predict(X_test_scaled)

# Plot feature importance
from xgboost import plot_importance
plot_importance(xgb_clf)
plt.show()

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Under Sampled Scaled Data and Untuned XGBoost Classifier

In [ ]:
# Define the model
xgb_rus_clf = XGBClassifier(random_state=1)

# Fit the model
xgb_rus_clf.fit(X_undersampled, y_undersampled)

# Predict using the model
train_pred = xgb_rus_clf.predict(X_train_scaled)
test_pred = xgb_rus_clf.predict(X_test_scaled)

# Plot feature importance
plot_importance(xgb_rus_clf)
plt.show()

# Generate classification report
# Print scores
print("Balanced Accuracy Scores")
print("--------------------------------------------------------")
x = balanced_accuracy_score(y_train, train_pred)
y = balanced_accuracy_score(y_test, test_pred)
print(balanced_accuracy_score(y_train, train_pred),'training')
print(balanced_accuracy_score(y_test, test_pred),'testing')
print(round((x-y), 16),'variance')

# Print classification reports
print("--------------------------------------------------------")
print(f"Classification Report")
report = classification_report(y_test, test_pred, zero_division=1)
print(report)
print("--------------------------------------------------------")

## Under Sampled Scaled Data with XGBoost Parameter Tuning

In [ ]:
# Try the following values for learning_rate
rate = np.arange(0.01, 1.1, 0.01)
models = {'train_score': [], 'test_score': [], 'learning_rate': []}

# Loop through each value in learning_rates
for r in rate:
    # Initialize the classifier with parameter variables
    xgb_rus_clf = XGBClassifier(learning_rate = r, tree_method='approx', random_state=1)

    # Fit the undersampled data the new model
    xgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = xgb_rus_clf.predict(X_train_scaled)
    test_pred = xgb_rus_clf.predict(X_test_scaled)

    # Calculate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['learning_rate'].append(r)

# Create a dataframe from the models dictionary with learning_rate as the index
models_df = pd.DataFrame(models).set_index('learning_rate')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(5))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 5
top_learning_rates = models_df.sort_values(by='test_score', ascending=False).head(5).index.tolist()

# Display the top 5 
top_learning_rates

In [ ]:
# Plot the results
models_df.plot()

In [ ]:
# Try the following values for n_estimators
estimators = range(100, 200, 1)
models = {'train_score': [], 'test_score': [], 'n_estimators': []}

# Loop through each value in n_estimators
for n in estimators:
    # Initialize the classifier with parameter variables
    xgb_rus_clf = XGBClassifier(n_estimators = n, tree_method='approx', random_state=1)

    # Fit the undersampled data the new model
    xgb_rus_clf.fit(X_undersampled, y_undersampled)

    # Make predictions
    train_pred = xgb_rus_clf.predict(X_train_scaled)
    test_pred = xgb_rus_clf.predict(X_test_scaled)

    # Calculate balanced accuracy scores
    train_score = balanced_accuracy_score(y_train, train_pred)
    test_score = balanced_accuracy_score(y_test, test_pred)

    # Append scores
    models['train_score'].append(train_score)
    models['test_score'].append(test_score)
    models['n_estimators'].append(n)

# Create a dataframe from the models dictionary with n_estimators as the index
models_df = pd.DataFrame(models).set_index('n_estimators')

# Display df
display(models_df.sort_values(by='test_score', ascending=False).head(5))

In [ ]:
# Sort the dataframe by test_score in descending order and get the top 5
top_n_estimators = models_df.sort_values(by='test_score', ascending=False).head(5).index.tolist()

# Display the top 5
top_n_estimators

In [ ]:
# Plot the results
models_df.plot()

In [2]:
# Stock list
stock_list = [
                'XLK',
                'XLV',
                'XLE',
                'VNQ',
                'XLF',
                'XLB',
                'XLU',
                'XLI',
                'XLP',
                'XLY',
                'XTL',
              ]

# Define dictionary
stock_df = {}

# Loop through stocks 
for stock in stock_list:
    # Read the data into DataFrames
    df = pd.read_csv(f'Resources/{stock}.csv')
    
    # Trim to last 10 years
    stock_df[stock] = df.iloc[-2520:]
    
    # Confirm shape
    print(f'{stock}',stock_df[stock].shape)

XLK (2520, 7)
XLV (2520, 7)
XLE (2520, 7)
VNQ (2520, 7)
XLF (2520, 7)
XLB (2520, 7)
XLU (2520, 7)
XLI (2520, 7)
XLP (2520, 7)
XLY (2520, 7)
XTL (2520, 7)


In [3]:
# Access a specific DataFrame
stock_df['XLK']

,Date,Open,High,Low,Close,Adj Close,Volume
3897,2014-06-20,38.200001,38.220001,38.009998,38.090000,33.428440,5427200
3898,2014-06-23,38.049999,38.200001,38.020000,38.200001,33.524986,3767100
3899,2014-06-24,38.180000,38.380001,37.910000,38.000000,33.349468,8471300
3900,2014-06-25,37.919998,38.209999,37.880001,38.150002,33.481102,5111900
3901,2014-06-26,38.180000,38.200001,37.889999,38.110001,33.446007,11352300
...,...,...,...,...,...,...,...
6412,2024-06-18,231.350006,232.169998,230.490005,231.410004,231.004745,4338900
6413,2024-06-20,232.449997,232.589996,228.039993,228.809998,228.409302,6621500
6414,2024-06-21,228.860001,229.759995,227.369995,228.410004,228.009995,6780400
6415,2024-06-24,225.639999,226.660004,222.360001,222.419998,222.419998,6874300
